In [1]:
import sys
sys.path.append("..")
import django
django.setup()

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_raw = pd.read_csv('data.csv')

_df = df_raw[["item_id", "output"]]
df = _df.pivot_table(index='item_id', columns="output", aggfunc=len, fill_value=0)

_df = df_raw[["user_id", "output"]]
dfa = _df.pivot_table(index='user_id', columns="output", aggfunc=len, fill_value=0)

## Weighting scheme
Based on https://github.com/jmgirard/mReliability/wiki/Weighting-scheme

In [4]:
def identity_kernel(x, y, q):
    return float(x == y)

def linear_kernel(x, y, q):
    return 1 - abs(x - y) / (q - 1)

def quadratic_kernel(x, y, q):
    return 1 - (x - y)**2 / (q - 1)**2


def get_weights(q, kernel):
    return np.array([
        [
            kernel(x, y, q) for x in range(q)
        ] for y in range(q)
    ])

## Observed agreement
Based on https://github.com/jmgirard/mReliability/wiki/Agreement-or-accuracy

In [5]:
def observed_agreement(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)
    
    w = get_weights(q, weights_kernel)
    
    r_star = df.dot(w)
    
    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum()/N
    return po

In [6]:
observed_agreement(df)

0.378021978021978

## Bennett et al.'s S score
Based on https://github.com/jmgirard/mReliability/wiki/Bennett-et-al.'s-S-score

In [7]:
def s_score(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)
    
    w = get_weights(q, weights_kernel)
    
    r_star = df.dot(w)
    
    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum()/N
    
    pc = w.sum() / q**2
    
    S = (po - pc) / (1 - pc)
    return S

In [8]:
s_score(df)

0.2225274725274725

In [9]:
def cohens_kappa(df, dfa, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum() / N

    p = dfa.div(dfa.sum(axis=1), axis=0)
    r, q = p.shape

    pbar = p.sum(axis=0) / r

    rpbar = r * np.array(pbar).reshape(1, q).T * np.array(pbar).reshape(1, q)
    pg = np.array(p).T.dot(np.array(p))
    s2 = (pg - rpbar) / (r - 1)

    pbarplus = np.array(pbar).reshape(1, q).T * np.array(pbar).reshape(1, q)
    pc = (w * (pbarplus - s2/r)).sum()

    k = (po - pc) / (1 - pc)
    return k

In [10]:
cohens_kappa(df, dfa)

0.21704246783787523

## Gwet's gamma coefficient
Based on https://github.com/jmgirard/mReliability/wiki/Gwet's-gamma-coefficient

In [11]:
def gwets_gamma(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum() / N

    Tw = w.sum()

    pi = df.div(df.sum(axis=1), axis=0).sum() / N

    pc = (pi*(1 - pi)).values.sum() * Tw / (q*(q-1))

    gamma = (po - pc) / (1 - pc)
    return gamma

In [12]:
gwets_gamma(df)

0.22561415081662817

## Krippendorff's alpha coefficient
Based on https://github.com/jmgirard/mReliability/wiki/Krippendorff's-alpha-coefficient

In [17]:
def krippendorffs_alpha(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po2 = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum()/N
    
    rdash = df.sum(axis=1).sum() / N

    epsilon = 1 / (N*rdash)
    po = po2 * (1-epsilon) + epsilon

    pi = df.div(df.sum(axis=1), axis=0).sum() / N

    pc = (w * np.array(pi).reshape(1, q).T * np.array(pi).reshape(1, q)).sum()

    alpha = (po - pc) / (1 - pc)
    return alpha

In [18]:
krippendorffs_alpha(df)

0.21557405653322695

## Scott's pi coefficient (Fleiss)
based on https://github.com/jmgirard/mReliability/wiki/Scott's-pi-coefficient

In [21]:
def scotts_pi(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum() / N

    pik = df.div(df.sum(axis=1), axis=0).sum() / N

    pc = (w * np.array(pik).reshape(1, q).T * np.array(pik).reshape(1, q)).sum()

    pi = (po - pc) / (1 - pc)
    return pi

In [22]:
scotts_pi(df)

0.20993070442195524